In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "/content/gdrive/MyDrive/dataset/"


In [ ]:
df_users_full = pd.read_csv(path + "users.csv")
df_users1 = df_users_full[["user_id","functional_unit","department"]]
df_users = df_users1[df_users1.functional_unit == "2 - ResearchAndEngineering"]
df_users = df_users[(df_users.department != "1 - Research")]
df_users = pd.DataFrame(df_users)

In [ ]:
df_users = df_users.dropna(axis = 0)
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3991 entries, 8 to 17333
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          3991 non-null   object
 1   functional_unit  3991 non-null   object
 2   department       3991 non-null   object
dtypes: object(3)
memory usage: 124.7+ KB


In [ ]:
df_device = pd.read_csv(path + "device.csv")

In [ ]:
df_file = pd.read_csv(path + "file.csv")

In [ ]:
df_logon = pd.read_csv(path + "logon.csv")

In [ ]:
df_psychometric = pd.read_csv(path + "psychometric.csv")
df_psychometric = df_psychometric[['employee_name','user_id','O','C','E', 'A', 'N']]

In [ ]:
df_users_clean = df_users.rename(columns= {'user_id':'user'}, inplace=False)

In [ ]:
df_logon_users = pd.merge(df_logon, df_users_clean, on = 'user')
df_logon_users_clean = df_logon_users.drop(columns=['functional_unit', 'department'])


In [ ]:
df_device_users = pd.merge(df_device, df_users_clean, on='user')
df_device_users_clean = df_device_users.drop(columns=['functional_unit','department'])

In [ ]:
df_device_users = pd.merge(df_device, df_users_clean, on = 'user')
df_device_users_clean = df_device_users.drop(columns = ['functional_unit', 'department'])


In [ ]:
df_file_users = pd.merge(df_file, df_users_clean, on = 'user')
df_file_users_clean = df_file_users.drop(columns = ['functional_unit', 'department'])


In [ ]:
df_psychometric_users = pd.merge(df_psychometric, df_users, on = 'user_id')
df_psychometric_users_clean = df_psychometric_users[['employee_name', 'user_id','O', 'C', 'E','A','N']]


In [ ]:
df_logon_users_clean['date'] = pd.to_datetime(df_logon_users_clean['date'])

In [ ]:
df_logon_users_clean['time'] = df_logon_users_clean['date'].dt.time

In [ ]:
df_user_logon = df_logon_users_clean.loc[df_logon_users_clean['activity'] == 'Logon']

In [ ]:
df_user_logon['hour'] = pd.to_datetime(df_user_logon['date'], format='%H:%M').dt.hour

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_user_logon_stats = df_user_logon.groupby('user')['time'].agg([min,max]).reset_index()

In [ ]:
df_logon_mode = df_user_logon.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [ ]:
df_logon_mean = df_user_logon.groupby('user')['hour'].mean().reset_index()
df_logon_mean['hour'].dtype
df_logon_mean['hour'] = pd.to_datetime(df_logon_mean['hour'], format='%H').dt.time


In [ ]:
df_user_logon_stats['mode'] = df_logon_mode['time']
df_user_logon_stats['mean'] = df_logon_mean['hour']

In [ ]:
df_user_logoff = df_logon_users_clean.loc[df_logon_users_clean['activity'] == 'Logoff']

In [ ]:
df_user_logoff['date'] = pd.to_datetime(df_user_logoff['date'])
df_user_logoff['time'] = df_user_logoff['date'].dt.time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:

df_user_logoff['hour'] = pd.to_datetime(df_user_logoff['date'], format='%H:%M').dt.hour

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_user_logoff_stats = df_user_logoff.groupby('user')['time'].agg([min,max]).reset_index()

In [ ]:
df_user_logoff.groupby('user')['time'].agg(pd.Series.mode).reset_index()

,user,time
0,AAN0077,16:56:00
1,AAW0103,16:14:00
2,ACE0265,16:06:00
3,ACL0394,17:15:00
4,ADR0362,16:45:00
...,...,...
225,XTM0246,17:00:00
226,YJT0368,16:45:00
227,YSB0779,16:45:00
228,ZBL0379,19:30:00


In [ ]:
df_logoff_mode = df_user_logoff.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [ ]:
df_logoff_mean = df_user_logoff.groupby('user')['hour'].mean().reset_index()
df_logoff_mean['hour'] = df_logoff_mean['hour'].astype(int)
df_logoff_mean['hour'] = pd.to_datetime(df_logoff_mean['hour'], format='%H').dt.time


In [ ]:
df_user_logoff_stats['mode'] = df_logoff_mode['time']
df_user_logoff_stats['mean'] = df_logoff_mean['hour']

In [ ]:
df_device_users_clean['time'] = pd.to_datetime(df_device_users_clean['date']).dt.time

In [ ]:
df_device_conn = df_device_users_clean.loc[df_device_users_clean['activity'] == 'Connect']
df_device_disconn = df_device_users_clean.loc[df_device_users_clean['activity'] == 'Disconnect']


In [ ]:
df_device_conn_stats = df_device_conn.groupby('user')['time'].agg([min, max]).reset_index()

In [ ]:
df_device_conn_stats_1 = df_device_conn.groupby('user')

In [ ]:
df_conn_mode = df_device_conn.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [ ]:
df_device_conn['hour'] = pd.to_datetime(df_device_conn['date']).dt.hour

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_conn_mean = df_device_conn.groupby('user')['hour'].mean().reset_index()
df_conn_mean['hour'] = df_conn_mean['hour'].astype(int)
df_conn_mean['hour'] = pd.to_datetime(df_conn_mean['hour'],format="%H").dt.time

In [ ]:
df_device_conn_stats['mode'] = df_conn_mode['time']
df_device_conn_stats['mean'] = df_conn_mean['hour']

In [ ]:
df_device_disconn_stats = df_device_disconn.groupby('user')['time'].agg([min,max]).reset_index()

In [ ]:
df_dconn_mode = df_device_disconn.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()



In [ ]:
df_device_disconn['hour'] = pd.to_datetime(df_device_disconn['date']).dt.hour


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_dconn_mean = df_device_disconn.groupby('user')['hour'].mean().reset_index()
df_dconn_mean['hour'] = df_dconn_mean['hour'].astype(int)
df_dconn_mean['hour'] = pd.to_datetime(df_dconn_mean['hour'], format='%H').dt.time


In [ ]:
df_dconn_mean

,user,hour
0,AJQ0376,14:00:00
1,AJR0231,11:00:00
2,AOD0066,12:00:00
3,ARH0777,12:00:00
4,BCP0247,12:00:00
5,BDS0396,13:00:00
6,BMS0057,15:00:00
7,BZK0095,11:00:00
8,CAE0080,12:00:00
9,CGH0088,12:00:00


In [ ]:
df_device_disconn_stats['mode'] = df_dconn_mode['time']
df_device_disconn_stats['mean'] = df_dconn_mean['hour']

In [ ]:
df_file_users_clean['date2'] = pd.to_datetime(df_file_users_clean['date']).dt.date

In [ ]:
df_files_per_day = df_file_users_clean.groupby(['user', 'date2']).size().reset_index()

In [ ]:
df_files_per_day.rename(columns={0:'transfers_per_day'}, inplace=True)


In [ ]:
df_files_max_per_day = df_files_per_day.groupby('user')['transfers_per_day'].agg(max).reset_index()
df_files_max_per_day.rename(columns={'transfers_per_day': "max_transfers_per_user"}, inplace=True)

In [ ]:
df_files_mode_per_day = df_files_per_day.groupby('user')['transfers_per_day'].agg(lambda x: x.value_counts().index[0]).reset_index()
df_files_mode_per_day.rename(columns={'transfers_per_day': "mode_transfers_per_user"}, inplace=True)


In [ ]:
df_files_mode_per_day

,user,mode_transfers_per_user
0,AJQ0376,486
1,AJR0231,90
2,AOD0066,30
3,ARH0777,36
4,BCP0247,72
5,BDS0396,72
6,BMS0057,36
7,BZK0095,486
8,CAE0080,36
9,CGH0088,18


In [ ]:
df_files_stats = df_files_mode_per_day

In [ ]:
df_files_stats_new = pd.DataFrame()
df_files_stats_new['user'] = df_files_stats['user']
df_files_stats_new['mode_trasfers_per_user'] = df_files_stats['mode_transfers_per_user']
df_files_stats_new['max_transfers_per_user'] = df_files_max_per_day['max_transfers_per_user']


In [ ]:
df_user_pc = df_logon_users_clean.groupby(['user','pc',]).agg(pc_visits_per_user_total = pd.NamedAgg(column = 'pc', aggfunc = 'count')).reset_index()
df_user_pc['count'] = df_user_pc.groupby(['user'])['pc'].transform('nunique')
df_user_pc = df_user_pc.drop(['pc', 'pc_visits_per_user_total'], axis=1)
df_user_pc = df_user_pc.drop_duplicates()

In [ ]:
df_user_pc

,user,count
0,AAN0077,1
1,AAW0103,1
2,ACE0265,1
3,ACL0394,2
5,ADR0362,1
...,...,...
632,XTM0246,1
633,YJT0368,2
635,YSB0779,1
636,ZBL0379,1


In [ ]:
# import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [ ]:
df_user_pc_count = np.array(df_user_pc['count'])

In [ ]:
df_user_pc_ct = df_user_pc_count.reshape(-1,1)

In [ ]:

model = pickle.load(open('/content/user_pc_ct.pkl', 'rb'))
graph_a_score = model.decision_function(df_user_pc_ct)
threat = model.predict(df_user_pc_ct)


In [ ]:
#user pc
#forest = IsolationForest(bootstrap=False, contamination=0.1,max_features=1.0,
#                          max_samples='auto',n_estimators=100, n_jobs=1, random_state=None,
#                          verbose=0)
# # forest.fit(df_user_pc_ct)


In [ ]:
# import pickle 
# with open('user_pc_ct.pkl','wb') as file :
#   pickle.dump(forest, file)

In [ ]:
# graph_a_score = forest.decision_function(df_user_pc_ct)
# print(graph_a_score[1:10])

In [ ]:
graph_result = pd.DataFrame()
graph_result['user'] = df_user_pc['user']
graph_result['ascore'] = graph_a_score
graph_result['threat']= threat
graph_result.to_csv('user_pc_ct.csv', index=False)
graph_result


,user,ascore,threat
0,AAN0077,0.088238,1
1,AAW0103,0.088238,1
2,ACE0265,0.088238,1
3,ACL0394,0.002247,1
5,ADR0362,0.088238,1
...,...,...,...
632,XTM0246,0.088238,1
633,YJT0368,0.002247,1
635,YSB0779,0.088238,1
636,ZBL0379,0.088238,1


In [ ]:
outliers = graph_result.loc[graph_result['ascore'] < 0]
print(outliers)

        user    ascore  threat
14   AJR0231 -0.244303      -1
54   ALC0100 -0.181540      -1
96   ARH0777 -0.178799      -1
110  BGZ0902 -0.278668      -1
154  CGH0088 -0.178799      -1
167  CQS0899 -0.245808      -1
205  DNS0397 -0.234844      -1
218  EER0383 -0.194948      -1
254  FED0275 -0.260518      -1
401  LWB0078 -0.365125      -1
489  PYT0264 -0.220159      -1
580  WJP0386 -0.279193      -1
592  WMP0272 -0.205776      -1


In [ ]:
def dtt2timestamp(dtt):
  time_in_sec = (dtt.hour*60 + dtt.minute) * 60 + dtt.second
  return time_in_sec

In [ ]:
df_user_logon_stats_sec = df_user_logon_stats


In [ ]:
min_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['min']]
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mean']]

In [ ]:
df_user_logon_stats_sec['min_ts'] = min_ts
df_user_logon_stats_sec['max_ts'] = max_ts
df_user_logon_stats_sec['mode_ts'] = mode_ts
df_user_logon_stats_sec['mean_ts'] = mean_ts

In [ ]:
df_user_logon_stats_sec.drop(['min','max','mode','mean'], axis=1)

,user,min_ts,max_ts,mode_ts,mean_ts
0,AAN0077,27900,29580,29580,25200
1,AAW0103,25200,26280,26100,25200
2,ACE0265,28800,30300,29700,28800
3,ACL0394,32400,60757,33300,32400
4,ADR0362,27000,28080,28080,25200
...,...,...,...,...,...
225,XTM0246,27900,29520,29460,25200
226,YJT0368,27000,68459,27900,25200
227,YSB0779,27000,28020,27900,25200
228,ZBL0379,29700,52585,30600,36000


In [ ]:
df_user_logoff_stats_sec = df_user_logoff_stats


In [ ]:
min_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['min']] 
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mean']]

In [ ]:
df_user_logoff_stats_sec['min_ts'] = min_ts
df_user_logoff_stats_sec['max_ts'] = max_ts
df_user_logoff_stats_sec['mode_ts'] = mode_ts
df_user_logoff_stats_sec['mean_ts'] = mean_ts

In [ ]:
df_user_logoff_stats_sec.drop(['min', 'max','mode','mean'], axis=1)

,user,min_ts,max_ts,mode_ts,mean_ts
0,AAN0077,60420,62100,60960,57600
1,AAW0103,58320,59400,58440,57600
2,ACE0265,57900,59400,57960,57600
3,ACL0394,33235,63000,62100,57600
4,ADR0362,60120,61200,60300,57600
...,...,...,...,...,...
225,XTM0246,60480,62100,61200,57600
226,YJT0368,27293,68732,60300,54000
227,YSB0779,60180,61200,60300,57600
228,ZBL0379,69420,71100,70200,68400


In [ ]:
df_log_on_off_stats = pd.DataFrame()

df_log_on_off_stats['user'] = df_user_logon_stats_sec['user']
df_log_on_off_stats['on_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['on_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['on_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['on_mean_ts'] = df_user_logon_stats_sec['mean_ts']
df_log_on_off_stats['off_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['off_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['off_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['off_mean_ts'] = df_user_logon_stats_sec['mean_ts']

In [ ]:
df_log_on_off_stats.dtypes

user           object
on_min_ts       int64
on_max_ts       int64
on_mode_ts      int64
on_mean_ts      int64
off_min_ts      int64
off_max_ts      int64
off_mode_ts     int64
off_mean_ts     int64
dtype: object

In [ ]:
log_stats = df_log_on_off_stats.drop(['user'], axis = 1)
log_stats_matrix = np.matrix(log_stats)
print(log_stats_matrix)

[[27900 29580 29580 ... 29580 29580 25200]
 [25200 26280 26100 ... 26280 26100 25200]
 [28800 30300 29700 ... 30300 29700 28800]
 ...
 [27000 28020 27900 ... 28020 27900 25200]
 [29700 52585 30600 ... 52585 30600 36000]
 [25200 26760 26100 ... 26760 26100 25200]]


In [ ]:
df_log_on_off_stats.columns

Index(['user', 'on_min_ts', 'on_max_ts', 'on_mode_ts', 'on_mean_ts',
       'off_min_ts', 'off_max_ts', 'off_mode_ts', 'off_mean_ts'],
      dtype='object')

In [ ]:
# 

model2 = pickle.load(open('/content/log_stats_matrix.pkl', 'rb'))
log_ascore = model2.decision_function(log_stats_matrix)
threat = model2.predict(log_stats_matrix)


In [ ]:
# # counting the values
# df22 = pd.Series(forest.predict(log_stats_matrix))
# # df22 = df22.map({1:0, -1:1})
# # print(df22.value_counts())

In [ ]:
# import pickle 
# with open('log_stats_matrix.pkl','wb') as file :
#   pickle.dump(forest, file)

In [ ]:
df_user_log_result = pd.DataFrame()
df_user_log_result['user'] = df_user_logoff_stats_sec['user']
df_user_log_result['ascore'] = log_ascore
df_user_log_result['threat'] = threat
df_user_log_result.to_csv('user_log_result.csv', index=False)
print(df_user_log_result)


        user    ascore  threat
0    AAN0077  0.044850       1
1    AAW0103  0.114800       1
2    ACE0265  0.166565       1
3    ACL0394  0.030387       1
4    ADR0362  0.154255       1
..       ...       ...     ...
225  XTM0246  0.059540       1
226  YJT0368  0.028291       1
227  YSB0779  0.200618       1
228  ZBL0379  0.118023       1
229  ZIF0269  0.100618       1

[230 rows x 3 columns]


In [ ]:
df_user_log_result.loc[df_user_log_result['ascore'] < 0]

,user,ascore,threat
14,AMR0656,-0.003612,-1
29,BKA0340,-0.017450,-1
43,BZV0067,-0.006930,-1
56,CQS0899,-0.112123,-1
65,DJS0406,-0.022337,-1
73,EER0383,-0.019264,-1
77,FED0275,-0.092863,-1
83,GFM0250,-0.038445,-1
86,GJP0098,-0.015209,-1
95,HML0060,-0.033975,-1


In [ ]:
df_device_conn_stats_sec = df_device_conn_stats
con_min_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['min']]
con_max_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['max']]
con_mode_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['mode']]
con_mean_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['mean']]

df_device_conn_stats_sec['min_ts'] = con_min_ts
df_device_conn_stats_sec['max_ts'] = con_max_ts
df_device_conn_stats_sec['mode_ts'] = con_mode_ts
df_device_conn_stats_sec['mean_ts'] = con_mean_ts
df_device_conn_stats_sec = df_device_conn_stats_sec.drop(['min', 'max','mode','mean'], axis=1)

In [ ]:
df_device_disconn_stats_sec = df_device_conn_stats
discon_min_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['min']]
discon_max_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['max']]
discon_mode_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['mode']]
discon_mean_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['mean']]

df_device_disconn_stats_sec['min_ts'] = discon_min_ts
df_device_disconn_stats_sec['max_ts'] = discon_max_ts
df_device_disconn_stats_sec['mode_ts'] = discon_mode_ts
df_device_disconn_stats_sec['mean_ts'] = discon_mean_ts
df_device_disconn_stats_sec = df_device_disconn_stats_sec.drop(['min', 'max','mode','mean'], axis=1)


In [ ]:
df_device_full = pd.DataFrame()
df_device_full['user'] = df_device_conn_stats['user']

In [ ]:
df_device_full['con_min_ts'] = df_device_conn_stats_sec['min_ts']
df_device_full['con_max_ts'] = df_device_conn_stats_sec['max_ts']
df_device_full['con_mode_ts'] = df_device_conn_stats_sec['mode_ts']
df_device_full['con_mean_ts'] = df_device_conn_stats_sec['mean_ts']

# disconnect stats
df_device_full['dcon_min_ts'] = df_device_disconn_stats_sec['min_ts']
df_device_full['dcon_max_ts'] = df_device_disconn_stats_sec['max_ts']
df_device_full['dcon_mode_ts'] = df_device_disconn_stats_sec['mode_ts']
df_device_full['dcon_mean_ts'] = df_device_disconn_stats_sec['mean_ts']

# files per day stats
df_device_full['file_mode'] = df_files_stats_new['mode_trasfers_per_user']
df_device_full['file_max'] = df_files_stats_new['max_transfers_per_user']

In [ ]:
device_full_matrix = df_device_full.drop(['user'],axis=1)
device_params = np.matrix(device_full_matrix)
device_params[:10]

matrix([[30401, 68694, 59963, 43200, 30401, 68694, 59963, 43200,   486,
           486],
        [27100, 56918, 47889, 36000, 27100, 56918, 47889, 36000,    90,
           252],
        [27693, 55895, 28260, 36000, 27693, 55895, 28260, 36000,    30,
            72],
        [27756, 59689, 29770, 39600, 27756, 59689, 29770, 39600,    36,
           144],
        [ 4870, 84764, 43474, 39600,  4870, 84764, 43474, 39600,    72,
           198],
        [33007, 61926, 35672, 39600, 33007, 61926, 35672, 39600,    72,
           234],
        [ 1185, 76882, 59905, 43200,  1185, 76882, 59905, 43200,    36,
           234],
        [28882, 59022, 47075, 39600, 28882, 59022, 47075, 39600,   486,
           486],
        [16018, 83814, 30456, 39600, 16018, 83814, 30456, 39600,    36,
           198],
        [29154, 57723, 32112, 39600, 29154, 57723, 32112, 39600,    18,
           198]])

In [ ]:
df_device_full.columns

Index(['user', 'con_min_ts', 'con_max_ts', 'con_mode_ts', 'con_mean_ts',
       'dcon_min_ts', 'dcon_max_ts', 'dcon_mode_ts', 'dcon_mean_ts',
       'file_mode', 'file_max'],
      dtype='object')

In [ ]:
# # device (-user)
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)

# forest.fit(device_params)


In [ ]:
# import pickle 
# with open('device_params.pkl','wb') as file :
#   pickle.dump(forest, file)

In [ ]:
# 

model3 = pickle.load(open('/content/device_params.pkl', 'rb'))
log_ascore = model3.decision_function(device_params)
threat = model3.predict(device_params)


In [ ]:
df_device_file_full_result = pd.DataFrame()

df_device_file_full_result['user'] = df_device_full['user']
df_device_file_full_result['ascore'] = log_ascore
df_device_file_full_result['threat'] = threat

df_device_file_full_result.to_csv('device_file_full_result.csv', index=False)

In [ ]:
df_device_file_full_result.loc[df_device_file_full_result['ascore'] < 0] 

,user,ascore,threat
6,BMS0057,-0.068229,-1
12,CSD0242,-0.058123,-1
37,QLC0248,-0.033315,-1
40,SBM0063,-0.035570,-1
46,ZBL0379,-0.000063,-1


In [ ]:
psychometric_matrix = df_psychometric_users_clean.drop(['user_id', 'employee_name'], axis = 1)
psychometric_params = np.matrix(psychometric_matrix)

In [ ]:
# # psycho ds 
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)
# forest.fit(psychometric_params)

In [ ]:
# import pickle 
# with open('psychometric_params.pkl','wb') as file :
#   pickle.dump(forest, file)

In [ ]:
# 

model4 = pickle.load(open('psychometric_params.pkl', 'rb'))
# df2=pd.DataFrame()
log_ascore = model4.decision_function(psychometric_params)
threat = model4.predict(psychometric_params)


In [ ]:
df_psychometric_result = pd.DataFrame()

df_psychometric_result['user'] = df_psychometric_users_clean['user_id']
df_psychometric_result['ascore'] = log_ascore
df_psychometric_result['threat']= threat
df_psychometric_result.to_csv('psychometric_result.csv', index=False)
df_psychometric_result

,user,ascore,threat
0,HBB0090,0.045387,1
1,HBB0090,0.045387,1
2,HBB0090,0.045387,1
3,HBB0090,0.045387,1
4,HBB0090,0.045387,1
...,...,...,...
3986,GGK0375,-0.030363,-1
3987,GGK0375,-0.030363,-1
3988,GGK0375,-0.030363,-1
3989,GGK0375,-0.030363,-1


In [ ]:
df_psychometric_result.loc[df_psychometric_result['ascore'] < 0]

,user,ascore,threat
203,JAB0249,-0.003142,-1
204,JAB0249,-0.003142,-1
205,JAB0249,-0.003142,-1
206,JAB0249,-0.003142,-1
207,JAB0249,-0.003142,-1
...,...,...,...
3986,GGK0375,-0.030363,-1
3987,GGK0375,-0.030363,-1
3988,GGK0375,-0.030363,-1
3989,GGK0375,-0.030363,-1


In [ ]:
df = pd.merge(df_log_on_off_stats, df_user_pc, on='user')

In [ ]:
df_1 = pd.merge(df, df_psychometric, left_on = 'user', right_on = 'user_id')

In [ ]:
df_1.head()

,user,on_min_ts,on_max_ts,on_mode_ts,on_mean_ts,off_min_ts,off_max_ts,off_mode_ts,off_mean_ts,count,employee_name,user_id,O,C,E,A,N
0,AAN0077,27900,29580,29580,25200,27900,29580,29580,25200,1,Allistair Akeem Nichols,AAN0077,42,15,32,22,29
1,AAW0103,25200,26280,26100,25200,25200,26280,26100,25200,1,Alexa Allegra Walton,AAW0103,39,42,47,39,28
2,ACE0265,28800,30300,29700,28800,28800,30300,29700,28800,1,Amos Carson Emerson,ACE0265,26,23,14,19,40
3,ACL0394,32400,60757,33300,32400,32400,60757,33300,32400,2,Adam Clark Lindsey,ACL0394,25,44,18,19,17
4,ADR0362,27000,28080,28080,25200,27000,28080,28080,25200,1,Amir Damian Rosario,ADR0362,32,42,35,38,26


In [ ]:
df_final = df_1.drop(['employee_name', 'user_id'], axis=1)


In [ ]:
df_all_parameters = df_final

In [ ]:
df_all_parameters_input = df_all_parameters.drop(['user'], axis = 1)

In [ ]:
print(df_all_parameters_input)

     on_min_ts  on_max_ts  on_mode_ts  on_mean_ts  ...   C   E   A   N
0        27900      29580       29580       25200  ...  15  32  22  29
1        25200      26280       26100       25200  ...  42  47  39  28
2        28800      30300       29700       28800  ...  23  14  19  40
3        32400      60757       33300       32400  ...  44  18  19  17
4        27000      28080       28080       25200  ...  42  35  38  26
..         ...        ...         ...         ...  ...  ..  ..  ..  ..
225      27900      29520       29460       25200  ...  18  22  29  34
226      27000      68459       27900       25200  ...  35  23  44  35
227      27000      28020       27900       25200  ...  46  16  19  40
228      29700      52585       30600       36000  ...  15  44  42  28
229      25200      26760       26100       25200  ...  20  17  46  26

[230 rows x 14 columns]


In [ ]:
# # psycho logon ds
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)
# forest.fit(df_all_parameters_input)

In [ ]:
# import pickle 
# with open('all_parameters_input.pkl','wb') as file :
#   pickle.dump(forest, file)

In [ ]:
# 

model5 = pickle.load(open('all_parameters_input.pkl', 'rb'))
# df2=pd.DataFrame()
log_ascore = model5.decision_function(df_all_parameters_input)
threat = model5.predict(df_all_parameters_input)


In [ ]:
df_all_parameters_result = pd.DataFrame()

df_all_parameters_result['user'] = df_final['user']
df_all_parameters_result['ascore'] =log_ascore
df_all_parameters_result['threat'] = threat
print(df_all_parameters_result)
df_all_parameters_result.to_csv('all_parameters_result.csv', index=False)

        user    ascore  threat
0    AAN0077  0.106260       1
1    AAW0103  0.046323       1
2    ACE0265  0.083103       1
3    ACL0394  0.018016       1
4    ADR0362  0.110909       1
..       ...       ...     ...
225  XTM0246  0.068278       1
226  YJT0368  0.029436       1
227  YSB0779  0.095632       1
228  ZBL0379  0.052035       1
229  ZIF0269  0.050711       1

[230 rows x 3 columns]
